In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')

100%|██████████| 3/3 [00:01<00:00,  2.08it/s]


In [4]:
for df in [df_hist,df_new]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])

In [6]:
df_hist['purchase_date'] = df_hist['purchase_date'].map(lambda x: str(x)[:10])
df_new['purchase_date'] = df_new['purchase_date'].map(lambda x: str(x)[:10])

In [7]:
df_hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10,1.0,16,37


### 124 same date feature

In [13]:
df_hist['cnt'] = 1
df_new['cnt'] = 1
auth1_hist = df_hist[df_hist.authorized_flag=='Y']
auth0_hist = df_hist[df_hist.authorized_flag=='N']

hist_cnt = df_hist.groupby([key, 'purchase_date'])['cnt'].sum().reset_index()
new_cnt = df_new.groupby([key, 'purchase_date'])['cnt'].sum().reset_index()

auth1_cnt = auth1_hist.groupby([key, 'purchase_date'])['cnt'].sum().reset_index()
auth0_cnt = auth0_hist.groupby([key, 'purchase_date'])['cnt'].sum().reset_index()

hist_cnt.head()

In [14]:
hist_cnt = hist_cnt.groupby(key)['cnt'].max().reset_index().rename(columns={'cnt':'hist_same_date_purchase_max'})
new_cnt = new_cnt.groupby(key)['cnt'].max().reset_index().rename(columns={'cnt':'new_same_date_purchase_max'})
auth0_cnt = auth0_cnt.groupby(key)['cnt'].max().reset_index().rename(columns={'cnt':'auth0_same_date_purchase_max'})

auth0_cnt.head()

,card_id,auth0_same_date_purchase_max
0,C_ID_00007093c1,3
1,C_ID_0001238066,2
2,C_ID_0001506ef0,1
3,C_ID_0001793786,6
4,C_ID_000183fdda,3


In [17]:
df_train = df_train.merge(hist_cnt ,how='left', on=key).merge(new_cnt ,how='left', on=key).merge(auth0_cnt ,how='left', on=key)
df_test = df_test.merge(hist_cnt ,how='left', on=key).merge(new_cnt ,how='left', on=key).merge(auth0_cnt ,how='left', on=key)
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_same_date_purchase_max,new_same_date_purchase_max,auth0_same_date_purchase_max
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,6,3.0,1.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,7,2.0,2.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,3,1.0,1.0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,5,1.0,NaN
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,8,3.0,3.0


In [18]:
feat_no = '124_sam_'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
    if not(col.count('max')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)